In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import os
import time
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
os.environ['PATH']+= r'C:/SeleniumDrivers'

In [3]:
driver=webdriver.Chrome()
driver.get('https://www.bigbasket.com/pc/snacks-branded-foods/biscuits-cookies/?nc=nb')

In [4]:
elements=driver.find_elements(By.CSS_SELECTOR,'a.h-full')

In [5]:
for i,j in enumerate(elements):
    print(j.text)
    print(i)
    print("........................")

Britannia
Marie Gold Biscuit - Crunchy, Light, Zero Trans Fat, Ready To Eat
0
........................
Parle
Gluco Biscuits - Parle-G
1
........................
Britannia
Bourbon Cream Biscuit - Chocolate Flavor
2
........................
Britannia
Good Day - Cashew Cookies
3
........................
Britannia NutriChoice
Digestive Zero (No Maida & Zero Added Sugar),
4
........................
Britannia NutriChoice
Digestive High Fibre Biscuits,
5
........................
Britannia
Treat - Jim Jam Biscuits
6
........................
Britannia
50-50 Maska Chaska
7
........................
Cadbury Oreo
Vanilla Flavour Cream Sandwich Biscuit
8
........................
Britannia
Good Day Butter Cookies
9
........................
Nestle
KitKat Share & Snap 2x3 Fingers Wafer Bar
10
........................
Parle
Krackjack
11
........................
Cadbury Oreo
Chocolate Flavour Cream Sandwich Biscuit, Jumbo Pack
12
........................
Sunfeast
Dark Fantasy - Choco Fills
13
...........

In [6]:
data_list=[]
def individual():
    try:
        prod_name = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'Description___StyledH-sc-82a36a-2'))).text
    except (NoSuchElementException, TimeoutException):
        prod_name = None

    try:
        price = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'Description___StyledTd-sc-82a36a-4'))).text
    except (NoSuchElementException, TimeoutException):
        price = None

    try:
        ingredients_label = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//span[text()="Ingredients"]')))
        ingredients_div = ingredients_label.find_element(By.XPATH, '../following-sibling::div[@class="overflow-hidden"]')
        ingredients_text = ingredients_div.text
    except (NoSuchElementException, TimeoutException):
        ingredients_text = None

    try:
        nutritional_label = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//span[text()="Nutritional Facts"]')))
        nutritional_div = nutritional_label.find_element(By.XPATH, '../following-sibling::div[@class="overflow-hidden"]')
        nutritional_text = nutritional_div.text
    except (NoSuchElementException, TimeoutException):
        nutritional_text = None

    print(prod_name, price, ingredients_text, nutritional_text)
    data_list.append({'Product Name': prod_name, 'Price': price, 'Ingredients': ingredients_text, 'Nutritional Facts': nutritional_text})


In [7]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException, TimeoutException, StaleElementReferenceException
import time

i = 0

for element in elements:
    try:
     
        for retry in range(3):  
            try:
                driver.implicitly_wait(5)
                element.click()
                break
            except StaleElementReferenceException:
                print("StaleElementReferenceException occurred. Retrying...")
                continue

        print("Clicked on the element")

    
        WebDriverWait(driver, 30).until(
            EC.invisibility_of_element_located((By.XPATH, '//a[@class="h-full"][@href="/pd/40276397/indisecrets-assorted-tulsi-infusions-with-4-exotic-flavours-sampler-pack-85-g/?nc=cl-prod-list&amp;t_pos_sec=3&amp;t_pos_item=3&amp;t_s=Assorted+Tulsi+Infusions+With+4+Exotic+Flavours"]'))
        )
        print("Element became invisible")


        WebDriverWait(driver, 30).until(lambda driver: len(driver.window_handles) > 1)
        print("New window opened")

        driver.switch_to.window(driver.window_handles[1])
        print("Switched to the new window")

        individual()
        WebDriverWait(driver, 30).until(lambda driver: len(driver.window_handles) > 1) 
        time.sleep(2)
        
        driver.close()
        print("Closed the new window")

  
        driver.switch_to.window(driver.window_handles[0])
        print("Switched back to the original window")

    except (WebDriverException, TimeoutException, StaleElementReferenceException) as e:
        print(f"Error occurred: {e}")
        continue


Error occurred: Message: element click intercepted: Element is not clickable at point (392, -6053)
  (Session info: chrome=120.0.6099.217)
Stacktrace:
	GetHandleVerifier [0x00007FF6A3002142+3514994]
	(No symbol) [0x00007FF6A2C20CE2]
	(No symbol) [0x00007FF6A2AC76AA]
	(No symbol) [0x00007FF6A2B188F3]
	(No symbol) [0x00007FF6A2B168ED]
	(No symbol) [0x00007FF6A2B140EC]
	(No symbol) [0x00007FF6A2B12EA5]
	(No symbol) [0x00007FF6A2B06A0F]
	(No symbol) [0x00007FF6A2B35FEA]
	(No symbol) [0x00007FF6A2B063B6]
	(No symbol) [0x00007FF6A2B36490]
	(No symbol) [0x00007FF6A2B528F6]
	(No symbol) [0x00007FF6A2B35D93]
	(No symbol) [0x00007FF6A2B04BDC]
	(No symbol) [0x00007FF6A2B05C64]
	GetHandleVerifier [0x00007FF6A302E16B+3695259]
	GetHandleVerifier [0x00007FF6A3086737+4057191]
	GetHandleVerifier [0x00007FF6A307E4E3+4023827]
	GetHandleVerifier [0x00007FF6A2D504F9+689705]
	(No symbol) [0x00007FF6A2C2C048]
	(No symbol) [0x00007FF6A2C28044]
	(No symbol) [0x00007FF6A2C281C9]
	(No symbol) [0x00007FF6A2C188C4

In [8]:
df=pd.DataFrame(data_list)
print(df)

                                         Product Name          Price  \
0   Parle Monaco Cheeslings - Classic, 2 x 150 g M...  Price: ₹117.6   
1   Dukes Waffy Choco Flavoured Rolls - Crunchy, C...    Price: ₹150   
2   Sunfeast Dark Fantasy - Yumfills, Rich Chocola...     Price: ₹82   
3   Parle Platina Nutricrunch Digestive Cookies - ...    Price: ₹135   
4   UNIBIC Fruit & Nut Cookies - Boosts Immunity, ...    Price: ₹480   
5          UNIBIC Assorted Cookies, 450 g (Pack of 6)    Price: ₹180   
6   Karachi Bakery Osmania Biscuits - Healthy, Cri...    Price: ₹152   
7   Parle Platina Hide N Seek Milano Finest Choco ...   Price: ₹98.6   
8   Malkist Chocolate Flavoured Cracker Biscuits -...    Price: ₹300   
9   Parle Platina Hide & Seek Black Bourbon Creme ...     Price: ₹87   
10  Dukes Waffy Strawberry Flavoured Wafer Rolls -...     Price: ₹60   

                                          Ingredients  \
0   Wheat Flour, Edible Vegetable Oils, Sugar, Sal...   
1   Sugar, Refined Wh

In [9]:
excel_filename='Atta, flours and sooji.xlsx'
df.to_excel(excel_filename,index=False)

In [12]:
import pandas as pd

# Sample nutritional values
nutritional_values = [
    "Amount Per 100g (approx), Carbohydrate 68.0g, of which sugars 3.3g, Protein 8.4g",
    "Per 100g: * Approximate Values Energy (kcal) 438 Protein (g) 5.4",
    "As Per - 100 g Energy - 489 kcal Protein - 6.5 g",
    "Per 100g Energy - 510Kcal, Protein - 7.4g",
    "Nutritional Values Per Serving Energy (Kcal) - 453 Protein (G) - 7 Fat (G) - 32 Carbohydrates (G) - 58"
]

# Define a function to extract nutritional values
def extract_nutritional_values(text):
    values = {}
    parts = [part.strip() for part in text.split(',')]
    for part in parts:
        if '-' in part:
            key, value = map(str.strip, part.split('-', 1))
            values[key] = value
    return values

# Create a list to store individual DataFrames
dfs = []

# Populate the list with DataFrames containing nutritional values
for text in nutritional_values:
    values = extract_nutritional_values(text)
    df = pd.DataFrame([values], columns=values.keys())
    dfs.append(df)

# Concatenate the individual DataFrames into a single DataFrame
result_df = pd.concat(dfs, ignore_index=True)

# Display the resulting DataFrame
print(result_df)


                                    As Per Per 100g Energy Protein  \
0                                      NaN             NaN     NaN   
1                                      NaN             NaN     NaN   
2  100 g Energy - 489 kcal Protein - 6.5 g             NaN     NaN   
3                                      NaN         510Kcal    7.4g   
4                                      NaN             NaN     NaN   

        Nutritional Values Per Serving Energy (Kcal)  
0                                                NaN  
1                                                NaN  
2                                                NaN  
3                                                NaN  
4  453 Protein (G) - 7 Fat (G) - 32 Carbohydrates...  


In [13]:
result_df

,As Per,Per 100g Energy,Protein,Nutritional Values Per Serving Energy (Kcal)
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,100 g Energy - 489 kcal Protein - 6.5 g,NaN,NaN,NaN
3,NaN,510Kcal,7.4g,NaN
4,NaN,NaN,NaN,453 Protein (G) - 7 Fat (G) - 32 Carbohydrates...


In [14]:
excel_filename='Apeifnii.xlsx'
result_df.to_excel(excel_filename,index=False)